## Maximum Subarray

* https://leetcode.com/problems/maximum-subarray/
***
* Time Complexity: O(n)
    - must traverse through the entire array to figure out the max
* Space Complexity: O(1)
    - only 2 variables are created and used. no recursion used either
***
* basically you keep a running max and another max that is the highest one you've seen
    - you update the running max as you move along with the current index in mind and then you see if that running max is the biggest you've seen so far and update accordingly

In [1]:
/**
 * @param {number[]} nums
 * @return {number}
 */
var maxSubArray = function(nums) {
    if (nums.length === 1) return nums[0];
    
    let max = nums[0];
    let runningMax = nums[0];
    
    for (let i = 1; i < nums.length; i++) {
        runningMax = Math.max(nums[i] + runningMax, nums[i]);
        max = Math.max(max, runningMax);
    }
    
    return max;
};

## Jump Game

* https://leetcode.com/problems/jump-game/
***
* Time Complexity: O(n)
    - just traverses through the array once
* Space Complexity: O(1)
    - only keeps a couple of variables
***
* similar structure to Kadane's algorithm
* my thought process:
    - as long as you can continue traversing through the array, you can move to the last index
    - so as long as your max steps > 0, then you can continue walking
    - so through each iteration
        * we subtract the max to simulate walking to the next index
        * we compare the current max with the number of steps at this index and we take whichever one is larger since it'll give us the best chance to continue walking towards the end
        * if our max steps ever reaches 0, then we've exhausted all of our steps and cannot reach the last index

In [1]:
/**
 * @param {number[]} nums
 * @return {boolean}
 */
var canJump = function(nums) {
  // since you start at the first index,
  // if steps = 0, you might not reach your goal
  // unless you have something like nums = [0]
  if (nums[0] === 0) return nums.length === 1;

  // initialized with first item in array
  let max = nums[0];
  for (let i = 1; i < nums.length - 1; i++) {
    // this is where we make our step
    max--;

    max = Math.max(max, nums[i]);

    // as long as we can keep moving forward, we can reach
    // the last index, so as long as max = 1
    if (max <= 0) return false;

  }

  return true;
};